In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [17]:
def load_dataset():
    train_dataset = h5py.File('../../Downloads/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('../../Downloads/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [124]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [125]:
X_train_orig.shape

(1080, 64, 64, 3)

In [126]:
Y_train_orig.shape

(1, 1080)

In [127]:
X_train=X_train_orig
Y_train=Y_train_orig.T

In [128]:
print (X_train.shape)
print (Y_train.shape)

(1080, 64, 64, 3)
(1080, 1)


In [129]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [130]:
Y_train=convert_to_one_hot(Y_train, 6)
print (Y_train.shape)


(6, 1080)


In [131]:
Y_train=Y_train.T
print (Y_train.shape)
X_train = X_train_orig/255

(1080, 6)


In [108]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32, shape=[None, 64,64,3])
y=tf.placeholder(tf.float32, shape=[None, 6])

In [109]:
W1=tf.get_variable("W1",shape=[4,4,3,8],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
W2=tf.get_variable("W2",shape=[2,2,8,16],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())

In [98]:
z1=tf.nn.conv2d(x,W1,strides=[1,1,1,1],padding='SAME')
a1=tf.nn.relu(z1)
p1=tf.nn.max_pool(a1,ksize=[1,8,8,1],strides=[1,8,8,1],padding='SAME')
z2=tf.nn.conv2d(p1,W2,strides=[1,1,1,1],padding='SAME')
a2=tf.nn.relu(z2)
p2=tf.nn.max_pool(a2,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')

p=tf.contrib.layers.flatten(p2)
z3=tf.contrib.layers.fully_connected(p,6,activation_fn=None)


In [101]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( labels=y, logits=z3))
optimizer=tf.train.AdamOptimizer(0.009).minimize(cost)

In [103]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(15):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))

Cost after epoch 0: 1.813791
Cost after epoch 1: 1.793399
Cost after epoch 2: 1.785255
Cost after epoch 3: 1.779145
Cost after epoch 4: 1.776160
Cost after epoch 5: 1.773582
Cost after epoch 6: 1.767942
Cost after epoch 7: 1.761678
Cost after epoch 8: 1.756579
Cost after epoch 9: 1.748854
Cost after epoch 10: 1.740276
Cost after epoch 11: 1.728979
Cost after epoch 12: 1.717050
Cost after epoch 13: 1.704452
Cost after epoch 14: 1.690390


In [102]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(300):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
        
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    #test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)

Cost after epoch 0: 1.891707
Cost after epoch 1: 1.799246
Cost after epoch 2: 1.788396
Cost after epoch 3: 1.790424
Cost after epoch 4: 1.790019
Cost after epoch 5: 1.788342
Cost after epoch 6: 1.785468
Cost after epoch 7: 1.782430
Cost after epoch 8: 1.777115
Cost after epoch 9: 1.772377
Cost after epoch 10: 1.766533
Cost after epoch 11: 1.762457
Cost after epoch 12: 1.756638
Cost after epoch 13: 1.751289
Cost after epoch 14: 1.742537
Cost after epoch 15: 1.735810
Cost after epoch 16: 1.724804
Cost after epoch 17: 1.714712
Cost after epoch 18: 1.700798
Cost after epoch 19: 1.686522
Cost after epoch 20: 1.669314
Cost after epoch 21: 1.652373
Cost after epoch 22: 1.633935
Cost after epoch 23: 1.612604
Cost after epoch 24: 1.589270
Cost after epoch 25: 1.564989
Cost after epoch 26: 1.538908
Cost after epoch 27: 1.517791
Cost after epoch 28: 1.490051
Cost after epoch 29: 1.463300
Cost after epoch 30: 1.439816
Cost after epoch 31: 1.411429
Cost after epoch 32: 1.389488
Cost after epoch 33:

Cost after epoch 268: 0.360328
Cost after epoch 269: 0.350077
Cost after epoch 270: 0.341422
Cost after epoch 271: 0.339527
Cost after epoch 272: 0.343347
Cost after epoch 273: 0.346453
Cost after epoch 274: 0.345971
Cost after epoch 275: 0.341548
Cost after epoch 276: 0.336006
Cost after epoch 277: 0.332547
Cost after epoch 278: 0.331476
Cost after epoch 279: 0.332218
Cost after epoch 280: 0.331943
Cost after epoch 281: 0.331129
Cost after epoch 282: 0.329578
Cost after epoch 283: 0.326795
Cost after epoch 284: 0.324303
Cost after epoch 285: 0.321876
Cost after epoch 286: 0.320254
Cost after epoch 287: 0.319937
Cost after epoch 288: 0.319820
Cost after epoch 289: 0.319318
Cost after epoch 290: 0.317973
Cost after epoch 291: 0.316129
Cost after epoch 292: 0.314318
Cost after epoch 293: 0.312569
Cost after epoch 294: 0.310798
Cost after epoch 295: 0.309342
Cost after epoch 296: 0.307993
Cost after epoch 297: 0.306928
Cost after epoch 298: 0.305980
Cost after epoch 299: 0.304950
0.898148

In [104]:
print (train_accuracy)

0.8981481


In [132]:
z1=tf.nn.conv2d(x,W1,strides=[1,1,1,1],padding='VALID')
a1=tf.nn.relu(z1)
p1=tf.nn.max_pool(a1,ksize=[1,8,8,1],strides=[1,8,8,1],padding='VALID')
z2=tf.nn.conv2d(p1,W2,strides=[1,1,1,1],padding='VALID')
a2=tf.nn.relu(z2)
p2=tf.nn.max_pool(a2,ksize=[1,4,4,1],strides=[1,4,4,1],padding='VALID')

p=tf.contrib.layers.flatten(p2)
z3=tf.contrib.layers.fully_connected(p,6,activation_fn=None)

In [133]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( labels=y, logits=z3))
optimizer=tf.train.AdamOptimizer(0.009).minimize(cost)

In [135]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(15):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    #test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)

Cost after epoch 0: 1.843714
Cost after epoch 1: 1.803115
Cost after epoch 2: 1.797971
Cost after epoch 3: 1.792009
Cost after epoch 4: 1.792710
Cost after epoch 5: 1.790609
Cost after epoch 6: 1.787047
Cost after epoch 7: 1.789328
Cost after epoch 8: 1.788612
Cost after epoch 9: 1.785432
Cost after epoch 10: 1.785769
Cost after epoch 11: 1.785109
Cost after epoch 12: 1.783922
Cost after epoch 13: 1.782001
Cost after epoch 14: 1.779765
0.28055555


In [136]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(30):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    #test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)

Cost after epoch 0: 1.798674
Cost after epoch 1: 1.813242
Cost after epoch 2: 1.788593
Cost after epoch 3: 1.786873
Cost after epoch 4: 1.787608
Cost after epoch 5: 1.789833
Cost after epoch 6: 1.787351
Cost after epoch 7: 1.782971
Cost after epoch 8: 1.780074
Cost after epoch 9: 1.777187
Cost after epoch 10: 1.774859
Cost after epoch 11: 1.772079
Cost after epoch 12: 1.767969
Cost after epoch 13: 1.764758
Cost after epoch 14: 1.763294
Cost after epoch 15: 1.758247
Cost after epoch 16: 1.749138
Cost after epoch 17: 1.744081
Cost after epoch 18: 1.740503
Cost after epoch 19: 1.730628
Cost after epoch 20: 1.721538
Cost after epoch 21: 1.715736
Cost after epoch 22: 1.704314
Cost after epoch 23: 1.691881
Cost after epoch 24: 1.682777
Cost after epoch 25: 1.668061
Cost after epoch 26: 1.654411
Cost after epoch 27: 1.640585
Cost after epoch 28: 1.622708
Cost after epoch 29: 1.608129
0.43703705


In [137]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(100):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    #test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)

Cost after epoch 0: 1.806876
Cost after epoch 1: 1.840260
Cost after epoch 2: 1.801538
Cost after epoch 3: 1.789351
Cost after epoch 4: 1.791415
Cost after epoch 5: 1.790464
Cost after epoch 6: 1.790143
Cost after epoch 7: 1.790160
Cost after epoch 8: 1.789428
Cost after epoch 9: 1.787779
Cost after epoch 10: 1.785928
Cost after epoch 11: 1.785399
Cost after epoch 12: 1.784654
Cost after epoch 13: 1.783269
Cost after epoch 14: 1.781588
Cost after epoch 15: 1.780046
Cost after epoch 16: 1.778568
Cost after epoch 17: 1.776016
Cost after epoch 18: 1.774358
Cost after epoch 19: 1.773032
Cost after epoch 20: 1.770297
Cost after epoch 21: 1.766062
Cost after epoch 22: 1.762790
Cost after epoch 23: 1.760878
Cost after epoch 24: 1.755832
Cost after epoch 25: 1.750611
Cost after epoch 26: 1.745499
Cost after epoch 27: 1.741089
Cost after epoch 28: 1.734257
Cost after epoch 29: 1.726961
Cost after epoch 30: 1.719583
Cost after epoch 31: 1.713959
Cost after epoch 32: 1.704611
Cost after epoch 33: